In [174]:
import requests
import re

we are getting the monster details we got from the webscraper.

In [175]:
master = "https://raw.githubusercontent.com/bshort95/AON_Scraper/refs/heads/main/monsterDeets.txt"
req = requests.get(master)
text = req.text

the following functions are for parsing the html data into usable data

In [176]:
def removebrax(text):
  tempText = ""
  tempText1 = ""
  write = True
  for x in text:
    if x == "<":
      write = False

    if(write):
      tempText = tempText + x;
    if x == ">":
      write = True
      if tempText[-1] != "|":
        tempText = tempText + "|"

  return tempText

In [188]:
def getName(text):
  text = removebrax(text)
  text = text.replace("|There is a Legacy version |here|.","")
  textList= text.split("|")
  if len(textList)>1:
    return textList[1]
  else:
    return ""

In [192]:
def getdesc(text, name):
  blob = text.split(">"+name+"<")
  if len(blob)>1:
    temp = removebrax(blob[1])[5:]
    temp1 = temp.split("Elite| | |Normal| | |Weak|Proficiency without Level|")[0]
    if "Recall Knowledge" in temp1:
      temp1 =  temp1.split("Recall Knowledge")[0]
    temp1 = temp1.replace("|","")
    if temp1 == "":
      temp1 = "This creature did not include a description"
    temp1 = temp1.replace("\"","")
    return "\"" + temp1 + "\""
  else:
    return "\"This creature did not include a description\""

In [179]:
def getid(text):
  temp = removebrax(text)
  temp1 = temp.split("|")[0]
  return temp1.split(":")[-2]

In [180]:
def getKnow(text):
  total = ""
  temp = removebrax(text)
  st = temp.find("Recall Knowledge")
  et = temp.find("Elite| | |")
  if(st > 0):
    ret = temp[st:et]
    ret = ret.replace("|"," ")
    ret = ret.replace("Recall Knowledge - ","")
    total = "\"" + ret + "\""
  return total

In [181]:
def getLevel(text):
  text1 = text[:text.find("<hr />")]
  text2 = removebrax(text1)
  text3 = text2[text2.find("Proficiency without Level|"):]
  block = text3.split("|")
  if len(block) >1:
    return block[2]
  return ""

In [191]:
def getAddInfo(text,name):
  text1 = text.split(">"+name+"<")[-1]
  text2 = removebrax(text1)
  text3 = text2[text2.rfind("|Source|"):]
  if "|Perception|" in text3:
    return ""
  text4 = "".join(text3.split("|")[4:])
  text4 = text4.replace("\"","")
  return "\"" + text4 + "\""

this is where we use the helper parser functions to compile the data into a useful csv format

In [187]:
def parseline(text):
  id = getid(text)
  name = getName(text)
  description = getdesc(text, name)
  knowledge = getKnow(text)
  level = getLevel(text)
  addInfo = getAddInfo(text, name)
  text = id + "," +"\""+ name+"\"" + "," + level + "," + description + "," + knowledge + "," + addInfo
  return text;

this is where we write to corpus data, which is what we use for the bm25 search.

In [193]:
temp = []
temp = text.splitlines()
header = "id,name,level,description,knowledge,addInfo\n"
f = open("/content/corpusData.csv", "w")
f.write(header)
for x in temp:
  if len(x) >6: # some edgecase ids with no content.
    f.write(parseline(x) + "\n")
f.close()
